In [2]:
import os
import pandas as pd
import json
import traceback

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
key=os.getenv('google_api_key')

In [6]:
pip install langchain-google-genai


Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1,google_api_key=key)

In [9]:
llm.invoke("what is the best mcq movie avengers or spiderman").content.strip()


'There\'s no single "best" MCQ (multiple-choice question) movie between Avengers and Spider-Man.  It depends entirely on what the questions are about.\n\n* **If the questions focus on overall cinematic scope, complex plots, and ensemble casts,** then *Avengers* (specifically *Avengers: Endgame*) would likely be a better choice for a challenging MCQ.\n\n* **If the questions focus on character development, relatable themes of adolescence and responsibility, and a more grounded narrative,** then a *Spider-Man* film (any of the recent MCU iterations) might be a better choice.\n\nTo create a good MCQ, you need to tailor the questions to the specific strengths of each franchise.'

In [10]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000260AB4D8940>, default_metadata=(), model_kwargs={})

In [11]:
pip install --upgrade langchain langchain-core langchain-google-genai



  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.67
    Uninstalling langchain-core-0.3.67:
      Successfully uninstalled langchain-core-0.3.67


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.32 requires langchain-core<0.2.0,>=0.1.41, but you have langchain-core 0.3.68 which is incompatible.
langchain-community 0.0.32 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.4.2 which is incompatible.


In [12]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import PyPDF2

In [14]:
TEMPLATE='''
TEXT:{text}
you are an expert MCQ maker . Given the above text , it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in{tone} tone.
make sure that no question is repeated and check all the questions to be conforming the text as well
make sure to format your response like RESPONSE_JSON below and use it like a guide. \
Ensure that to make {number} MCQs
### RESPONSE_JSON
{response_json}

'''


In [15]:
prompt_template=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [16]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    }
    
     
}

In [17]:
chain=LLMChain(llm=llm,prompt=prompt_template,output_key="quiz",verbose=False)

C:\Users\thouf\AppData\Local\Temp\ipykernel_8440\3679699836.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=llm,prompt=prompt_template,output_key="quiz",verbose=False)


In [18]:
TEMPLATE2='''
you are an expert in english grammar and writer . Given a multiple choice quiz for {subject} students.\
you need to evaluate the complexity of the quiz and goive a complete analysis of the quiz . Only use at max 50 wordsfor complexity
if the quiz is not as per the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the studentsabilities
Quiz_MCQs:
{quiz}

check from an expert English Writer of the above quiz:
'''


In [19]:
quiz_evaluation_template=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [20]:
quiz_evaluation_chain=LLMChain(llm=llm,prompt=quiz_evaluation_template,output_key="review",verbose=False)

In [21]:
connection_chain=SequentialChain(chains=[chain,quiz_evaluation_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=False)

In [22]:
file_path=r"C:\Users\thouf\mcqgeneratornew\data.txt"

In [23]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [24]:
TEXT

'In machine learning, deep learning focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be supervised, semi-supervised or unsupervised.[2]\n\nSome common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and b

In [25]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [26]:
def estimate_gemini_token_usage(prompt_inputs: dict, model_outputs: dict = None) -> int:
    input_tokens = 0
    for key, value in prompt_inputs.items():
        if isinstance(value, str):
            input_tokens += len(value.split())
        elif isinstance(value, dict):
            input_tokens += len(json.dumps(value).split())
        else:
            input_tokens += len(str(value).split())
    output_tokens = 0
    if model_outputs:
        for key, value in model_outputs.items():
            if isinstance(value, str):
                
                output_tokens += len(value.split())
            elif isinstance(value, dict):
                output_tokens += len(json.dumps(value).split())
            else:
                output_tokens += len(str(value).split())
    return input_tokens + output_tokens
if __name__ == "__main__":
    inputs = {
        "text": TEXT,
        "number": 3,
        "subject": "Machine learning",
        "tone": "simple",
        "response_json": json.dumps(RESPONSE_JSON)
    }
    try:
        print("\nStarting the MCQ generation and evaluation process...")
        response = connection_chain.invoke(inputs)
        total_tokens = estimate_gemini_token_usage(inputs, response)
        print(f"Estimated Total Tokens Used: {total_tokens}")
        print("\nGenerated Quiz (Raw LLM Output) ")
        quiz_output = response.get("quiz", "No quiz generated or output is not a string.")
  
        print(f"\nAn error occurred during chain invocation: {e}")
        traceback.print_exc()       
        try:
            parsed_quiz = json.loads(quiz_output)
            print(json.dumps(parsed_quiz, indent=2))
        except json.JSONDecodeError:
            print("Quiz output is not valid JSON. Raw output:")
    except Exception as e:    
        print(quiz_output)
        print("\n Quiz Review")
        print(response.get("review", "No review generated."))
   



Starting the MCQ generation and evaluation process...
Estimated Total Tokens Used: 866

Generated Quiz (Raw LLM Output) 
```json
{
  "1": {
    "mcq": "What is the defining characteristic of 'deep' learning in the context of neural networks?",
    "options": {
      "a": "Use of a single, large neuron",
      "b": "Employing multiple layers of artificial neurons",
      "c": "Focusing solely on supervised learning methods",
      "d": "Exclusively using recurrent neural network architectures"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of the following is NOT mentioned as a common deep learning network architecture in the provided text?",
    "options": {
      "a": "Convolutional Neural Networks (CNNs)",
      "b": "Recurrent Neural Networks (RNNs)",
      "c": "Support Vector Machines (SVMs)",
      "d": "Generative Adversarial Networks (GANs)"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "According to the text, how do current neural networks relate to the biolog

In [29]:
print(json.dumps(parsed_quiz, indent=2))

{
  "1": {
    "mcq": "What is the defining characteristic of 'deep' learning in the context of neural networks?",
    "options": {
      "a": "Use of a single, large neuron",
      "b": "Employing multiple layers of artificial neurons",
      "c": "Focusing solely on supervised learning methods",
      "d": "Exclusively using recurrent neural network architectures"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of the following is NOT mentioned as a common deep learning network architecture in the provided text?",
    "options": {
      "a": "Convolutional Neural Networks (CNNs)",
      "b": "Recurrent Neural Networks (RNNs)",
      "c": "Support Vector Machines (SVMs)",
      "d": "Generative Adversarial Networks (GANs)"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "According to the text, how do current neural networks relate to the biological function of the human brain?",
    "options": {
      "a": "They are highly accurate models of brain function.",
      "b": "

In [28]:
import json
import traceback

quiz_table_data=[]
quiz_output=response.get("quiz")

parsed_quiz=None
if quiz_output:
        
        # Step 1: Clean the string by removing markdown code block markers
        # .strip() removes leading/trailing whitespace (including newlines)
        # .replace() removes the markdown code block markers like ```json and ```
        clean_quiz = quiz_output.strip().replace("```json", "").replace("```", "")
        
        
         # Step 2: Attempt to parse the cleaned string into a Python dictionary
        # THIS IS THE CRUCIAL STEP: parsed_quiz WILL BE YOUR DICTIONARY
        parsed_quiz=json.loads(clean_quiz)

         # Step 3: ONLY proceed to iterate if parsed_quiz is successfully a dictionary
        # Check if parsed_quiz is indeed a dictionary before iterating
        if isinstance(parsed_quiz, dict):
           
            for mcq_id, mcq_details in parsed_quiz.items(): 
                mcq = mcq_details["mcq"]
                options = '|'.join(
                    [
                        f"{option}:{option_value}"
                        for option, option_value in mcq_details['options'].items()
                    ]
                )
                correct = mcq_details['correct']
                quiz_table_data.append({"MCQ": mcq, "Choice": options, "Correct": correct})

            print("Quiz data successfully processed into quiz_table_data:")
            print(json.dumps(quiz_table_data, indent=2)) 
 
    

Quiz data successfully processed into quiz_table_data:
[
  {
    "MCQ": "What is the defining characteristic of 'deep' learning in the context of neural networks?",
    "Choice": "a:Use of a single, large neuron|b:Employing multiple layers of artificial neurons|c:Focusing solely on supervised learning methods|d:Exclusively using recurrent neural network architectures",
    "Correct": "b"
  },
  {
    "MCQ": "Which of the following is NOT mentioned as a common deep learning network architecture in the provided text?",
    "Choice": "a:Convolutional Neural Networks (CNNs)|b:Recurrent Neural Networks (RNNs)|c:Support Vector Machines (SVMs)|d:Generative Adversarial Networks (GANs)",
    "Correct": "c"
  },
  {
    "MCQ": "According to the text, how do current neural networks relate to the biological function of the human brain?",
    "Choice": "a:They are highly accurate models of brain function.|b:They are inspired by biological systems but are not considered high-quality models of brain 

In [30]:
print(quiz_table_data)

[{'MCQ': "What is the defining characteristic of 'deep' learning in the context of neural networks?", 'Choice': 'a:Use of a single, large neuron|b:Employing multiple layers of artificial neurons|c:Focusing solely on supervised learning methods|d:Exclusively using recurrent neural network architectures', 'Correct': 'b'}, {'MCQ': 'Which of the following is NOT mentioned as a common deep learning network architecture in the provided text?', 'Choice': 'a:Convolutional Neural Networks (CNNs)|b:Recurrent Neural Networks (RNNs)|c:Support Vector Machines (SVMs)|d:Generative Adversarial Networks (GANs)', 'Correct': 'c'}, {'MCQ': 'According to the text, how do current neural networks relate to the biological function of the human brain?', 'Choice': 'a:They are highly accurate models of brain function.|b:They are inspired by biological systems but are not considered high-quality models of brain function.|c:They have no relation to biological systems.|d:They perfectly replicate the complexity of t

In [31]:
import pandas as pd 

In [34]:
quiz=pd.DataFrame(quiz_table_data)

In [35]:
quiz.to_csv("machinelearning.csv",index=False)